# Etapa 4: Experimentos adicionais

Esta etapa visa realizar experimentos adicionais a fim de aplicar variações no modelo e compará-los ao final.
Serão realizados 6 experimentos:
- Experimento 1: Fine-tuning do MobileNetV2
- Experimento 2: Utilização de outra rede - ResNet50
- Experimento 3: Utilização de outra rede - EfficientNetB0
- Experimento 4: Variação de dropout
- Experimento 5: Variação de learning rate
- Experimento 6: Variação de batch size

In [ ]:
# Importando funções compartilhadas do utils
from utils import build_model, build_generators, plot_history, ModelType

# Importações
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# Keras imports
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2, ResNet50, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# Definição de caminhos
DATA_DIR = "../data/FairFace"
SPLITS_DIR = "../splits"
MODEL_SAVE_DIR = "../models"
WEIGHTS_PATH = "../weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"

# Definição de parâmetros comuns (os mesmos do modelo base)
IMG_SIZE = (224, 224)
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 1e-4

# Carregar CSVs com splits processados
df_train = pd.read_csv(os.path.join(SPLITS_DIR, "train_split.csv"))
df_val = pd.read_csv(os.path.join(SPLITS_DIR, "val_split.csv"))
df_train['label'] = df_train['label'].astype(str)
df_val['label'] = df_val['label'].astype(str)
NUM_CLASSES = df_train['label'].nunique()

In [ ]:
# Geradores do mobilenetv2
mobilenet_train_gen, mobilenet_val_gen = build_generators(df_train, df_val, DATA_DIR, IMG_SIZE, BATCH_SIZE, mobilenet_preprocess)

# Experimento 1: **Fine-tuning** do MobileNetV2

In [ ]:
# Construção do modelo
fine_tuned_model = build_model(
    model_type=ModelType.MOBILENET,
    input_shape=IMG_SIZE,
    num_classes=NUM_CLASSES,
    dropout_rate=0.5,
    fine_tune_at=-20
)

fine_tuned_model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Treinar modelo
finetuned_history = fine_tuned_model.fit(
    mobilenet_train_gen,
    validation_data=mobilenet_val_gen,
    epochs=EPOCHS
)

# Salvar modelo
fine_tuned_model.save(os.path.join(MODEL_SAVE_DIR, "mobilenetv2_finetuned.keras"))

# Verificar resultados
plot_history(finetuned_history, title_suffix="(Fine-tuning MobileNetV2)")

# Experimento 2: MobileNetV2 com variação de **Dropout (0.2)**

In [ ]:
# Construção do modelo
dropout_model = build_model(ModelType.MOBILENET, IMG_SIZE, NUM_CLASSES, dropout_rate=0.2)
fine_tuned_model = build_model(
    model_type=ModelType.MOBILENET,
    input_shape=IMG_SIZE,
    num_classes=NUM_CLASSES,
    dropout_rate=0.5,
    fine_tune_at=-20
)

dropout_model.compile(
  optimizer=Adam(learning_rate=LEARNING_RATE),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

# Treinar modelo
dropout_history = dropout_model.fit(
  mobilenet_train_gen,
  validation_data=mobilenet_val_gen,
  epochs=EPOCHS
)

# Salvar modelo
dropout_model.save(os.path.join(MODEL_SAVE_DIR, "mobilenetv2_dropout02.keras"))

# Verificar resultados
plot_history(dropout_history, title_suffix="(Dropout 0.2)")

# Experimento 3: MobileNetV2 com variação de **learning rate (1e-3)**

In [ ]:
# Construção do modelo
learning_rate_model = build_model(ModelType.MOBILENET, IMG_SIZE, NUM_CLASSES, dropout_rate=0.5)
learning_rate_model.compile(
  optimizer=Adam(learning_rate=1e-3),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

# Treinar modelo
learning_rate_history = learning_rate_model.fit(
  mobilenet_train_gen,
  validation_data=mobilenet_val_gen,
  epochs=EPOCHS
)

# Salvar modelo
learning_rate_model.save(os.path.join(MODEL_SAVE_DIR, "mobilenetv2_lr1e3.keras"))

# Verificar resultados
plot_history(learning_rate_history, title_suffix="(LR = 1e-3)")

# Experimento 4: MobileNetV2 com variação de **Batch Size 64**

In [ ]:
# Construção do modelo
bs64_model = build_model(ModelType.MOBILENET, IMG_SIZE, NUM_CLASSES, dropout_rate=0.5)
bs64_model.compile(
  optimizer=Adam(learning_rate=LEARNING_RATE),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

# Treinar o modelo
bs64_history = bs64_model.fit(
  mobilenet_train_gen,
  validation_data=mobilenet_val_gen,
  epochs=EPOCHS
)

# Salvar o modelo
bs64_model.save(os.path.join(MODEL_SAVE_DIR, "mobilenetv2_bs64.keras"))

# Verificar resultados
plot_history(bs64_history, title_suffix="(Batch size 64)")

# Experimento 5: Utilização da rede **ResNet50**

Como esta rede possui uma nova arquitetura, precisaremos definir novos geradores.

In [ ]:
resnet_train_gen, resnet_val_gen = build_generators(df_train, df_val, DATA_DIR, IMG_SIZE, BATCH_SIZE, resnet_preprocess)

# Construção do modelo
resnet_model = build_model(ModelType.RESNET, IMG_SIZE, NUM_CLASSES, dropout_rate=0.5)
resnet_model.compile(
  optimizer=Adam(learning_rate=LEARNING_RATE),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

# Treinamento do modelo
resnet_history = resnet_model.fit(
  resnet_train_gen,
  validation_data=resnet_val_gen,
  epochs=EPOCHS
)

# Salvar modelo
resnet_model.save(os.path.join(MODEL_SAVE_DIR, "resnet50.keras"))

# Verificar resultados
plot_history(resnet_history, title_suffix="(ResNet50)")

# Experimento 6: Utilização da rede **EfficientNetB0**

In [ ]:
# Definição dos geradores
efficientnet_train_gen, efficientnet_val_gen = build_generators(df_train, df_val, DATA_DIR, IMG_SIZE, BATCH_SIZE, efficientnet_preprocess)

# Construção do modelo
efficientnet_model = build_model(ModelType.EFFICIENTNET, IMG_SIZE, NUM_CLASSES)
efficientnet_model.compile(
  optimizer=Adam(learning_rate=LEARNING_RATE),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

# Treinamento do modelo
efficientnet_history = efficientnet_model.fit(
  efficientnet_train_gen,
  validation_data=efficientnet_val_gen,
  epochs=EPOCHS
)

# Salvar modelo
efficientnet_model.save(os.path.join(MODEL_SAVE_DIR, "efficientnetb0.keras"))

# Verificar resultados
plot_history(efficientnet_history, title_suffix="(EfficientNetB0)")